# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By # localiza elementos e intens de sites
from selenium.webdriver.common.keys import Keys #usa techa do teclado

navegador = webdriver.Chrome()

# passo 1 entrar no google
navegador.get("https://www.google.com")

# passo 2 pesquisar a cotaçãp do dolar
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação dólar")
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# passo 3 pegar a cotação do dolar
cotacao_dolar = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

# passo 4 pegar a cotação do euro
navegador.get("https://www.google.com")
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação Euro")
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

# passo 5 pegar a cotação do ouro

navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [ ]:
# passo 6 atualizar minha base de dados com os novos valores
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)


- Atualizando os preços e o cálculo do Preço Final

In [ ]:
#tabela.loc[linha,"cotação"] = float(cotacao_dolar)

tabela.loc[tabela["Moeda"]=="Dólar","Cotação"]=float(cotacao_dolar)

#euro
tabela.loc[tabela["Moeda"]=="Euro","Cotação"]=float(cotacao_euro)


#Ouro
tabela.loc[tabela["Moeda"]=="Ouro","Cotação"]=float(cotacao_ouro)



# preço de compra igual ao preço original * cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]
# preço de venda igual ao preço original * margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

### Agora vamos exportar a nova base de preços atualizada

In [ ]:
tabela.to_excel("Produtos Novos.xlsx", index=False)
navegador.quit()